In [41]:
from psychopy.sound.backend_ptb import SoundPTB as Sound
from psychtoolbox import GetSecs, WaitSecs, hid
from psychopy import visual, core, event, prefs
import numpy as np
import pandas as pd
import random
import os

# prefs.hardware['audioLib'] = ['PTB', 'sounddevice', 'pyo','pygame']
# print(prefs.hardware['audioLib'])

ISI = 0.4
# TARGET = 130
N_SEQS = 3
SEQ_LEN = 10
# N_TARGETS_MIN = 15
# N_TARGETS_MAX = 25

pygame 2.1.2 (SDL 2.0.18, Python 3.6.15)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [46]:
def play_tone(ISI, tone_fpath):
    print('playing tone')
    now = GetSecs()
    snd = Sound(tone_fpath)
    snd.play(when = now + 0.1)
    print('waiting for keypress')
    event.waitKeys(keyList = ['return'], maxWait = 5)
    print('buffer time')
    WaitSecs(ISI - 0.1)

def get_target():
    target = random.choice([130, 200, 280])
    return(target)

# Get list of tone files to play
def get_tone_array(target):
    tones = [[130, 130],
             [130, 200],
             [130, 280],
             [200, 130],
             [200, 200],
             [200, 280],
             [280, 130],
             [280, 200],
             [280, 280]]
    tones.remove([target, target])
    tones = np.array(tones)
    return(tones)

def get_is_target(tones, stream, target):
    if stream == 'l':
        distractors = tones[:, 0]
    elif stream == 'r':
        distractors = tones[:, 1]
    is_target = [x == target for x in distractors]
    return(is_target)

def get_n_targets(n_targets_min, n_targets_max):
    n_targets = random.randint(n_targets_min, n_targets_max)
    return(n_targets)

def get_stream():
    stream = random.choice(['r', 'l'])
    return(stream)

def get_tone_fname(tone_array):
    left_freq = tone_array[0]
    right_freq = tone_array[1]
    fname = "tones/left_" + str(left_freq) + "_right_" + str(right_freq) + ".wav"
    return(fname)

def _replaceitem(x, target, no_targets):
    if x == target:
        if no_targets:
            return(0)
        else:
            return(1)
    else:
        return(2)

def get_tone_weights(stream, target, tones, no_targets):
    if stream == 'l':
        distractors = tones[:, 0]
    elif stream == 'r':
        distractors = tones[:, 1]
    weights = list(map(_replaceitem, distractors, [target] * len(distractors), [no_targets] * len(distractors)))
    return(weights)

def get_tone(tones, tone_id, markers, weights, no_target_weights, cannot_be_target):
    if cannot_be_target:
        i = random.choices(range(len(tones)), no_target_weights)[0]
    else:
        i = random.choices(range(len(tones)), weights)[0]
    tone = tones[i]
    is_target = tone_id[i]
    print(tone)
    print(is_target)
    return(tone, is_target)    

In [24]:
def broadcast(n_tones, var):
    if not isinstance(var, list):
        broadcasted_array = [var]*n_tones
    return(broadcasted_array)

def open_log(SUB_NUM, BLOCK_NUM):
    log = "../data/logs/sub-" + SUB_NUM + "_blk-" + BLOCK_NUM + ".log"
    if not os.path.isfile(log): # create log file if it doesn't exist
        print(f"Creating {log}")
        d = {
                'seed': [],
                'sub_num': [],
                'block_num': [],
                'seq_num': [],
                'stream': [],
                'target': [],
                'tone_num' : [],
                'left_freq' : [],
                'right_freq' : [],
                'is_target' : [],
            }
        df = pd.DataFrame(data = d)
        df.to_csv(log, mode='w', index = False)
    return(log)

def write_log(LOG, SEQ_LEN, seed, sub_num, block_num, seq_num, stream, target, tone_num, left_freq, right_freq, is_target):
    print("Writing to log file")
    d = {
        'seed': broadcast(SEQ_LEN, seed),
        'sub_num': broadcast(SEQ_LEN, sub_num),
        'block_num': broadcast(SEQ_LEN, block_num),
        'seq_num': broadcast(SEQ_LEN, seq_num),
        'stream': broadcast(SEQ_LEN, stream),
        'target': broadcast(SEQ_LEN, target),
        'tone_num' : tone_nums,
        'left_freq' : left_freq,
        'right_freq' : right_freq,
        'is_target' : is_target,
    }
    df = pd.DataFrame(data = d)
    df.to_csv(LOG, mode='a', header = False, index = False)

In [47]:
SUB_NUM = "0"
BLOCK_NUM = "0"
SEED = int(SUB_NUM + "0" + BLOCK_NUM)
random.seed(SEED)

LOG = open_log(SUB_NUM, BLOCK_NUM)

for seq_num in range(N_SEQS):
    # Randomize stream and target
    stream = get_stream()
    target = get_target()
    
    # Get weights for tones to select from
    tones = get_tone_array(target)
    weights = get_tone_weights(stream, target, tones, False)
    no_target_weights = get_tone_weights(stream, target, tones, True)
    tone_id = get_is_target(tones, stream, target)
    
    tone_nums = []
    left_freq = []
    right_freq = []
    is_targets = []

    cannot_be_target = True
    for tone_num in range(1, SEQ_LEN + 1):
        tone, is_target = get_tone(tones, tone_id, weights, no_target_weights, cannot_be_target)
        tone_fname = get_tone_fname(tone)
        cannot_be_target = is_target
        play_tone(ISI, tone_fname)
        break

        tone_nums.append(tone_num)
        left_freq.append(tone[0])
        right_freq.append(tone[1])
        is_targets.append(is_target)
        break
        
#     write_log(LOG,
#               SEQ_LEN,
#               SEED, 
#               SUB_NUM,
#               BLOCK_NUM,
#               seq_num,
#               stream, 
#               target, 
#               tone_nums,
#               left_freq, 
#               right_freq, 
#               is_targets)

[130 130]
False
playing tone
waiting for keypress
buffer time
[280 280]
False
playing tone
waiting for keypress
buffer time
[130 280]
False
playing tone
waiting for keypress
buffer time


KeyboardInterrupt: 

In [54]:
import RTBox
import time
startTime = time.time()
snd = Sound('tones/left_130.wav')
snd.play()
WaitSecs(3)
# (secs, btns) = box.secs(5)
executionTime = (time.time() - startTime)
print('Execution time in seconds: ' + str(executionTime))

Execution time in seconds: 3.0017106533050537


In [60]:
import RTBox
box = RTBox.RTBox()
boxSecs()

OSError: 
Darwin-20.6.0-x86_64-i386-64bit
Python 3.6.15
pySerial 3.5
No RTBox serial port found. Make sure FTDI driver is installed

In [60]:
np.shape(tones[:])

(8, 2)

In [49]:
is_target

array([ True,  True, False, False, False, False, False, False])

In [98]:
tone

array([280, 280])

In [59]:
np.shape(is_target)

(8,)

In [65]:
np.shape(is_target)
np.shape(tones)

(8, 2)

In [63]:
# is_target = np.transpose(is_target)
np.vstack((tones, is_target))

ValueError: all the input array dimensions for the concatenation axis must match exactly, but along dimension 1, the array at index 0 has size 2 and the array at index 1 has size 8